In [ ]:
# Bibliotecas utilizadas no estudo
# Professor Odemir Depieri Jr  -- Data Viking
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pyod.utils.data import generate_data

from warnings import filterwarnings , catch_warnings

In [ ]:
# Criando a base de dados
dados_x, dados_y = generate_data(n_train=1000, train_only=True, n_features=2)

eixo_x = dados_x[:,[0]].reshape(-1,1)
eixo_y = dados_x[:,[1]].reshape(-1,1)

tabela = pd.DataFrame(eixo_x, columns=['Eixo x'])
tabela['Eixo y'] = eixo_y
tabela['classe'] = dados_y

tabela.head()

In [ ]:
# Verificando a distribuição das classes
tabela['classe'].value_counts( normalize=True ) * 100

In [ ]:
# Plotando a distribuição dos valores

plt.figure(figsize=(10,8))

sns.scatterplot(x='Eixo x', y='Eixo y', hue='classe', data=tabela)

plt.title('Analise de distriuição dos valores' , fontsize=18, loc = 'left')

plt.show

In [ ]:
# Dados estatísticos
tabela.describe().transpose()

In [ ]:
# Verificando a distribuição dos dados em boxplot
plt.figure(figsize=(10,6))

plt.subplot(1,2,1)
plt.title('Análise eixo x')
sns.boxplot(data = tabela , y = 'Eixo x')

plt.subplot(1,2,2)
plt.title('Análise eixo y')
sns.boxplot(data = tabela , y = 'Eixo y')

In [ ]:
# Calculos para extrair outliers

q1 = np.percentile( tabela['Eixo x'] , 25 , interpolation='midpoint' )
q3 = np.percentile( tabela['Eixo x'] , 75 , interpolation='midpoint' )

# Intervalo do quartil
iqr = q3 - q1

print(f'Q1: {q1}')
print(f'Q3: {q3}')
print(f'IQR: {iqr}')

In [ ]:
# Verificando valores que precisam sair ou serem tratados
limite_superior = tabela['Eixo x'] >= q3 + 1.5 * iqr
limite_inferior = tabela['Eixo x'] <= q1 - 1.5 * iqr

print(f'Valores que são outliers:')
print(np.where(limite_superior))
print(np.where(limite_inferior))

In [ ]:
# Aplicação

menor = q1 - 1.5 * iqr
maior = q3 + 1.5 * iqr

dados_superior = np.where( tabela['Eixo x'] >= maior )
dados_inferior = np.where( tabela['Eixo x'] <= menor )

print(f'Serão retirados/tratados {dados_superior[0].shape} registros porque sao maiores que o q3')
print(f'Serão retirados/tratados {dados_inferior[0].shape} registros porque sao menores que o q1')

In [ ]:
# Criando nova base sem os outliers
nova_base = tabela.drop( dados_superior[0] )
nova_base = nova_base.drop( dados_inferior[0] )

In [ ]:
print(f'Antes haviam {tabela.shape[0]} registros')
print(f'Agora há {nova_base.shape[0]} registros')
print(f'Foram retirados {tabela.shape[0] - nova_base.shape[0]} registros')

In [ ]:
# Novo box plot com a nova base
plt.figure(figsize=(10,6))

plt.subplot(1,2,1)
plt.title('Análise eixo x')
sns.boxplot(data = nova_base , y = 'Eixo x')

plt.subplot(1,2,2)
plt.title('Análise eixo y')
sns.boxplot(data = nova_base , y = 'Eixo y')